In [13]:
#=
Step 0: Open recording or generate signal
=#

# To Generate Signal:
include("../signal_generator/tone.jl")

freq = 1500;          # Frequency of Tone (in Hz)
amp = 1;            # Amplitude of Tone
duration = 10;       # Duration of Tone (in seconds)
sample_rate = 32000.0;
tone_sig, n = tone(duration, amp, freq, sample_rate);

include("../signal_generator/generate_sig.jl")
using DSP.Windows: hanning, rect, bartlett
az_gt = -90;        # Ground Truth Azimuth Angle (in degrees)
az_gt2 = 120;
c0 = 1500;          # Speed of Medium (in m/s)
NFFT = 2^11;
noverlap = Int(NFFT * (3 // 4));
sig1, sample_rate = simulate_sensor_signal(tone_sig, sample_rate, sensors_underwater,
                         NFFT, noverlap, hanning, az_gt, c0);
sig2, sample_rate = simulate_sensor_signal(tone_sig, sample_rate, sensors_underwater, 
                        NFFT, noverlap, hanning, az_gt2, c0); 
new_sig = sig1;

Start Simulating Signal
Signal has size: (320000, 1)


  1.501132 seconds (404.54 k allocations: 1.463 GiB, 10.73% gc time, 4.37% compilation time)
Now Generated Signal has size: (320000, 40)
Start Simulating Signal
Signal has size: (320000, 1)


  1.419054 seconds (180.56 k allocations: 1.453 GiB, 10.00% gc time)
Now Generated Signal has size: (320000, 40)


In [14]:
include("../utils/preprocess.jl")
function filter_freq_per_ch(new_sig)
    new_S = []
    for sig in eachcol(new_sig)
        S_interest = choose_freq(sig, freq, sample_rate);
        push!(new_S, S_interest);
    end
    # test_sig = Matrix{}(undef, size(new_sig, 2)) 
    new_S = mapreduce(permutedims, vcat, new_S);
    return new_S
end

new_S = filter_freq_per_ch(new_sig);

In [16]:
using Statistics
n_snapshots = 128;
Sy = cov(new_S[:,1:n_snapshots], dims=2)

40×40 Matrix{ComplexF64}:
  1224.47+0.0im           1224.47-4.31294e-14im  …   1154.69+365.012im
  1224.47+4.31294e-14im   1224.47+0.0im              1154.69+365.012im
  1224.47+4.31294e-14im   1224.47+4.31294e-14im      1154.69+365.012im
  1224.47+4.31294e-14im   1224.47+4.31294e-14im      1154.69+365.012im
  1224.47+4.31294e-14im   1224.47+4.31294e-14im      1154.69+365.012im
  -1206.0+170.96im        -1206.0+170.96im       …  -1188.24-198.289im
 -1218.08-172.613im      -1218.08-172.613im         -1097.22-525.887im
  1177.96+372.075im       1177.96+372.075im          999.926+702.023im
 -1215.81-187.837im      -1215.81-187.837im         -1090.53-539.565im
  1177.96+372.075im       1177.96+372.075im          999.926+702.023im
         ⋮                                       ⋱  
  -1167.9-290.748im       -1167.9-290.748im         -1014.68-622.33im
 -1203.75+186.046im      -1203.75+186.046im         -1190.62-183.393im
  1154.69-365.012im       1154.69-365.012im          1197.71+2.40807e-

In [15]:
include("../sensor.jl")
include("./cbf.jl")
using LinearAlgebra

sensors Loaded: (8,)
sensors1 Loaded: (25,)
sensors2 Loaded: (20,)
sensors_underwater loaded: (40,)


In [9]:
az_list = -180:180;
num_sensors = length(sensors_underwater);
A = Matrix{ComplexF64}(undef, num_sensors, length(az_list))
for (sensor_idx, sensor) in enumerate(sensors_underwater)
    for (az_idx, az) in enumerate(az_list)
        A[sensor_idx, az_idx] = vandermonde_weight(sensor, az, 90, freq, c0);
    end
end


In [ ]:
# function 

In [68]:
# SBL Algorithm
ϵ = 0.001; # Numerical Tolerance of Algorithm
γ_collection = Vector{Float64}(undef, length(az_list));
max_iter = 500;
K = 1;      # Number of Peaks
σ_noise = 0.1;
num_sensors = length(sensors_underwater);
# for iter = 1:max_iter

γ_collection, ___ = cbf(Sy, sensors_underwater, freq, c0);

for iter = 1:max_iter
    Γ = diagm(γ_collection);
    Σy = σ_noise .* I + A * Γ * A'; # Note: I is the identity Matrix

    # Start Finding new Amplitudes
    γ_new = Vector{ComplexF64}(undef, length(γ_collection));
    for (az_idx, az) in enumerate(az_list)
        a = A[:, az_idx];
        inv_y = inv(Σy);
        numo = tr(Sy * inv_y * a * a' * inv_y); # Is this PSD too by right?
        denom = a' * inv_y * a;         # Is this PSD by right?
        # println("Numerator: $(numo), Denomenator: $(denom)")
        γ_new[az_idx] = γ_collection[az_idx] .* abs(numo) / abs(denom);
    end
    γ_new = abs.(γ_new);

    # Find Peaks & Re-estimate Noise Parameter
    ___, az_max_idx = findmax(γ_new); # Technically Find local Peaks
    Am = A[:, az_max_idx];
    σ_noise = tr(I - Am * pinv(Am)) ./ (num_sensors - K);
    σ_noise = abs(σ_noise);

    err = norm(γ_new - γ_collection, 1) ./ norm(γ_collection, 1);
    println("At iter: $iter , Noise Value = $(σ_noise) w/ error: $err")
    # Stopping Criterion
    err < ϵ ? break : γ_collection = γ_new;
end

At iter: 1 , Noise Value = 1.0000000000000007 w/ error: 0.9881150740103903


At iter: 2 , Noise Value = 1.0000000000000007 w/ error: 0.9914736079626799


At iter: 3 , Noise Value = 1.0000000000000007 w/ error: 0.7133602525951463


At iter: 4 , Noise Value = 1.0000000000000007 w/ error: 0.39970763126262504


At iter: 5 , Noise Value = 1.0000000000000007 w/ error: 0.5825008816285114


At iter: 6 , Noise Value = 1.0000000000000007 w/ error: 0.4553946823063477


At iter: 7 , Noise Value = 1.0000000000000007 w/ error: 0.09694990273167471


At iter: 8 , Noise Value = 1.0000000000000007 w/ error: 0.00556809147423416


At iter: 9 , Noise Value = 1.0000000000000007 w/ error: 0.00019142472874529495


In [69]:
σ_noise

1.0000000000000007

In [62]:
# γ_new - γ_collection

361-element Vector{ComplexF64}:
 NaN + NaN*im
 NaN + NaN*im
 NaN + NaN*im
 NaN + NaN*im
 NaN + NaN*im
 NaN + NaN*im
 NaN + NaN*im
 NaN + NaN*im
 NaN + NaN*im
 NaN + NaN*im
     ⋮
 NaN + NaN*im
 NaN + NaN*im
 NaN + NaN*im
 NaN + NaN*im
 NaN + NaN*im
 NaN + NaN*im
 NaN + NaN*im
 NaN + NaN*im
 NaN + NaN*im

In [50]:
Am * pinv(Am)

40×40 Matrix{ComplexF64}:
      0.025+0.0im         …   0.00656315-0.0241231im
     -0.025-1.1962e-5im       -0.0065747+0.02412im
      0.025+2.3924e-5im       0.00658624-0.0241168im
     -0.025-3.5886e-5im      -0.00659777+0.0241137im
      0.025+4.78479e-5im      0.00660931-0.0241105im
   0.015137-0.0198965im   …   -0.0152248-0.0198294im
  0.0129743-0.0213698im       -0.0172142-0.0181293im
  0.0114163+0.0222411im        0.0244581-0.00517699im
 -0.0130154+0.0213448im        0.0171792+0.0181624im
  0.0114376+0.0222302im        0.0244531-0.00520039im
           ⋮              ⋱  
  0.0246488+0.00417546im          0.0105-0.0226881im
 -0.0151666+0.019874im         0.0151953+0.019852im
 0.00658624+0.0241168im            0.025-2.3924e-5im
  0.0231626+0.00940708im       0.0151579-0.0198806im
  0.0215355-0.0126973im   …   -0.0065983-0.0241135im
 0.00298419-0.0248213im       -0.0231672-0.00939574im
 -0.0245847+0.00453772im     -0.00207558+0.0249137im
 -0.0231581-0.00941816im      -0.0151674+0.

40×40 Matrix{ComplexF64}:
 0.1+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.1+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.1+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
    ⋮                             ⋱                        
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0